## Sentinel-2

#### Sentinel-2 is an Earth observation mission from the EU Copernicus Programme that systematically acquires optical imagery at high spatial resolution (up to 10 m for some bands). The mission is based on a constellation of two identical satellites in the same orbit, 180° apart for optimal coverage and data delivery. Together, they cover all Earth's land surfaces, large islands, inland and coastal waters every 3-5 days.

#### Sentinel-2A was launched on 23 June 2015 and Sentinel-2B followed on 7 March 2017. Both of the Sentinel-2 satellites carry an innovative wide swath high-resolution multispectral imager with 13 spectral bands. For more information on the Sentinel-2 platforms and applications, check out the [European Space Agency website](http://www.esa.int/Applications/Observing_the_Earth/Copernicus/Overview4).

#### Digital Earth Australia (DEA) applies corrections to Sentinel-2 satellite images to arrive at a [surface reflectance](https://cmi.ga.gov.au/ga_s2_m_nbart_1) product (see the [introduction to Digtial Earth Australia](../Beginners_guide/02_DEA.ipynb) section for more information on the surface reflectance corrections). Surface reflectance provides standardised optical datasets by using robust physical models to correct for variations in image radiance values due to atmospheric properties, as well as sun and sensor geometry. The resulting stack of surface reflectance grids are consistent over space and time, which is instrumental in identifying and quantifying environmental change.

#### DEA provides two Sentinel-2 surface reflectance products:

1. **Sentinel-2 Definitive** (e.g. `s2a_ard_granule`): These products represent the 'definitive' source of high quality Sentinel-2 surface reflectance data, and are available from the beginning of the Sentinel-2 archive up to a delay of several weeks. 

2. **Sentinel-2 Near Real Time** (e.g. `s2a_nrt_granule`): These products are processed with best-available ancillary information and provided as a rolling 90 day archive of imagery which is typically available to load within approximately ~24 hours of a satellite overpass.

#### Both Sentinel-2 Definitive and Sentinel-2 Near Real Time products contain data processed to two surface reflectance corrections:

1. **NBAR** (e.g. `nbar_green`): NBAR stands for Nadir-corrected BRDF Adjusted Reflectance, where BRDF stands for Bidirectional reflectance distribution function.
    The approach involves atmospheric correction to compute surface-leaving radiance and bi-directional reflectance modelling to remove the effects of topography and angular variation in reflectance.

2. **NBAR-T** (e.g. `nbart_green`): Surface reflectance NBAR-T includes the terrain illumination reflectance correction and has the same features of NBAR, along with some additional features.


#### Sentinel-2 surface reflectance products have 13 spectral channels:

| Sentinel-2 bands | DEA band name | Band number | Central wavelength (nm) | Resolution (m) | Bandwidth (nm) |
| -----------------|---------------|-------------|-------------------------|----------------|----------------|
| Coastal aerosol | `nbar(t)_coastal_aerosol` | 1 | 443 | 60 | 20 |
| Blue | `nbar(t)_blue` | 2 | 490 | 10 | 65 |
| Green | `nbar(t)_green` | 3 | 560 | 10 | 35 |
| Red | `nbar(t)_red` | 4 | 665 | 10 | 30 |
| Vegetation red edge | `nbar(t)_red_edge_1` | 5 | 705 | 20 | 15 |
| Vegetation red edge | `nbar(t)_red_edge_2` | 6 | 740 | 20 | 15 |
| Vegetation red edge | `nbar(t)_red_edge_3` | 7 | 783 | 20 | 20 |
| NIR | `nbar(t)_nir_1` | 8 | 842 | 10 | 115 |
| Narrow NIR | `nbar(t)_nir_2` | 8A | 865 | 20 | 20 |
| Water vapour | N/A | 9 | 945 | 60 | 20 |
| SWIR - Cirrus | N/A | 10 | 1375 | 60 | 20 |
| SWIR | `nbar(t)_swir_2` | 11 | 1610 | 20 | 90 |
| SWIR | `nbar(t)_swir_3` | 12 | 2190 | 20 | 180 |

#### These bands cover the visible, near-infrared and short-wave infrared wave lengths.

!['Sentinel-2 spectral bands'](http://www.geosage.com/highview/figures/Sentinel2_Spectral_Bands.jpg)
     
> **Note**: There are a number of additional datasets that are also returned as part of Sentinel-2 queries.
These are a combination of datasets used in the correction process, and layers relating to pixel quality.

#### Load packages and connect to the datacube

In [ ]:
import sys
import datacube
import xarray as xr
from odc.ui import with_ui_cbk
from matplotlib import pyplot as plt

sys.path.append("./Scripts")
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_plotting import display_map
from datacube.utils.masking import make_mask

dc = datacube.Datacube(app="Sentinel_2")

#### We will load **Sentinel-2 Definitive** data from the Sentinel-2A and Sentinel-2B satellites using two methods. Firstly, we will use `dc.load()` to return a time series of satellite images from a single sensor. Secondly, we will load a time series using the `load_ard()` function, which is a wrapper function around the dc.load module. This function will load all the images from both Sentinel-2A and Sentinel-2B, combine them, and then apply a cloud mask. The returned `xarray.Dataset` will contain analysis ready images with the cloudy and invalid pixels masked out.

#### You can change any of the parameters in the `query` object below to adjust the location, time, projection, or spatial resolution of the returned datasets.

#### Sentinel-2 data is stored on file with a range of different coordinate reference systems or CRS (i.e. multiple UTM zones). The different satellite bands also have different resolutions (10 m, 20 m and 60 m).  Because of this, all Sentinel-2 queries need to include the following two query parameters:

* `output_crs`: This sets a consistent CRS that all Sentinel-2 data will be reprojected to, regardless of the UTM zone the individual image is stored in.
* `resolution`: This sets the resolution that all Sentinel-2 images will be resampled to. 

> Note: Be aware that setting `resolution` to the highest available resolution (i.e. `(-10, 10)`) will downsample the coarser resolution 20 m and 60 m bands, which may introduce unintended artefacts into your analysis.
It is typically best practice to set `resolution` to match the lowest resolution band being analysed. For example, if your analysis uses both 10 m and 20 m resolution bands, set `"resolution": (-20, 20)`.

In [ ]:
# Create a query object
query = {
    "x": (153.45, 153.47),
    "y": (-28.90, -28.92),
    "time": ("2018-01", "2018-02"),
    "output_crs": "EPSG:3577",
    "resolution": (-10, 10),
    "group_by": "solar_day",
}

#### The two **Sentinel-2 Definitive** products are:

* `s2a_ard_granule`
* `s2b_ard_granule`

>Note: Here we will load in a time-series of satellite images from only Sentinel-2A. To load in images from Sentinel-2B, change the `product` variable to `'s2b_ard_granule'`.

In [ ]:
ds = dc.load(product="s2a_ard_granule",
             progress_cbk=with_ui_cbk(),
             **query)

ds

#### The returned dataset contains all of the bands available for Sentinel-2. These include both `NBAR` and `NBAR-T` versions of the optical bands, along with [fmask](../Frequently_used_code/Masking_data.ipynb) (used for cloud masking) and other measurements (e.g. `azimuthal_exiting`, `azimuthal_incident`) that are used for generating the surface reflectance product.

#### Usually we are not interested in returning all the possible bands, but instead are only interested in a subset of these. If we wished to return only a few of `NBAR-T` optical bands, then we would pass a `measurements` parameter to `dc.load()` (or, alternatively, amend the initial `query` object to have a `measurements` parameter). 

In [ ]:
bands = ["nbart_blue", "nbart_green", "nbart_red"]

ds = dc.load(product="s2a_ard_granule",
             measurements=bands,
             progress_cbk=with_ui_cbk(),
             **query)

ds

#### Let's plot the facetted contents of this Sentinel-2 dataset:

In [ ]:
rgb(ds, col="time", col_wrap=6)

#### The often called agriculture RGB composite, uses bands SWIR-2, near-infrared and blue. It’s mostly used to monitor crop health, as both short-wave and near infrared bands are particularly good at highlighting dense vegetation, which appears dark green in the composite. SWIR measurements can help scientists estimate how much water is present in plants and soil, as water reflects SWIR light. Shortwave-infrared bands are also useful for distinguishing between snow, and ice, all of which appear white in visible light. Newly burned land reflects strongly in SWIR bands, making them valuable for mapping fire damage.

#### Exercise 5.1: Can you create an "agriculture RGB composite of the previous images.

> Tip: You might need to load new data specifying these bands. Remember that you can specify a `bands` parameter in the `rgb()` with the list of bands to map to the RGB channels.

In [ ]:
ds = ?

rgb(?)

#### Sentinel-2 ARD contains a variable called `fmask` with information relative to pixel quality. The `fmask` variable contains values from 0 to 5, using a similar encoding to Landsat.

| Value | Description |
|-------|-------------|
| 0 | Null |
| 1 | Valid |
| 2 | Cloud |
| 3 | Cloud shadow |
| 4 | Snow |
| 5 | Water |

#### Exercise 5.2: Load a new Dataset adding the `fmask` variable to the one in the previous exercise. Can you choose an area containing a water body and use the values in `fmask` to filter anything that is not classified as water in the images?

> Note: You can visualise the evolution of water bodies over a period of time. Would you know how to calculate the evolution of the surface or extent covered by water through time?

In [ ]:
ds = ?

#### Alternatively to the previous method, DEA offers the `load_ard()` function to load Sentinel-2 data more conveniently. This function will load images from both Sentinel-2A and Sentinel-2B, concatenate and sort the observations by time, and apply cloud masking.

#### For example:

In [ ]:
bands = ["nbart_blue", "nbart_green", "nbart_red"]

ds = load_ard(dc=dc,
              products=["s2a_ard_granule", "s2b_ard_granule"],
              measurements=bands,
              **query)

ds

In [ ]:
rgb(ds, col='time', col_wrap=6)

#### As you can see in the previous images, cloud masking in Sentinel-2 is not as reliable as in Landsat data. Improving the quality of cloud masking is a work in progress and you'll see an alternative solution later in this notebook.

#### Sentinel-2 Near Real Time (NRT): The surface reflectance processing workflow for Sentinel-2 Definitive products typically causes a delay before this data is available to load through the datacube.

#### To address this delay, DEA also provides a Sentinel-2 Near Real Time product which is available to load as early as ~24 hours after a satellite observation is made. Near Real Time data is processed using best-available ancillary information to provide atmospheric corrections, and delivered as a rolling 90 day archive of imagery.

#### The two Sentinel-2 NRT products are:

* `s2a_nrt_granule`
* `s2b_nrt_granule`

#### We can load Sentinel-2 NRT using either `dc.load()` or `load_ard()` using the methods demonstrated above:

In [ ]:
query = {
    "x": (153.45, 153.47),
    "y": (-28.90, -28.92),
    "measurements": ["nbart_blue", "nbart_green", "nbart_red"],
    "output_crs": "EPSG:3577",
    "resolution": (-10, 10),
    "group_by": "solar_day",
}

ds = load_ard(dc=dc,
              products=["s2a_nrt_granule", "s2b_nrt_granule"],
              mask_pixel_quality=False,
              **query)

ds

#### We can now plot the most recently acquired Sentinel-2 image in our dataset:

In [ ]:
# `index=-1` selects the final image in the dataset
rgb(ds, index=-1, size=8)

#### Exercise 5.3: Try to get the most recent image available for the place you choose in Australia.

In [ ]:
ds = ?

#### Exercise 5.4: Use the facetted plots to display the evolution of the selected area using the "agriculture" false colour representation.

> Hint: It might help to choose a sunny agricultural region to visualise the evolution of crops during the past 3 months.

In [ ]:
ds = ?

#### We are now going to see how to remove clouds of Sentinel2 images and how to create cloud free composites by combining several images.

#### We start by defining an area in Canberra:

In [ ]:
query = {
    'y': (-35.28, -35.34),
    'x': (149.12, 149.18),
    'time': ('2018-01', '2018-03'),
    'output_crs': 'EPSG:3577',
    'resolution': (-20, 20)
}

display_map(x=query['x'], y=query['y'])

#### We now load data from both Sentinel 2A and 2B datasets. We perform separate queries and then concatenate the data into a single dataset by interleaving the times so they are ordered.

> Note: This step can take several minutes to complete; please be patient. 

In [ ]:
# Load in Sentinel-2A and 2B data including fmask cloud masking band
s2a_ds = dc.load(
    product='s2a_ard_granule',
    measurements=['fmask', 'nbart_red', 'nbart_green', 'nbart_blue'],
    progress_cbk=with_ui_cbk(),
    **query)

s2b_ds = dc.load(
    product='s2b_ard_granule',
    measurements=['fmask', 'nbart_red', 'nbart_green', 'nbart_blue'],
    progress_cbk=with_ui_cbk(),
    **query)

# Combine Sentinel-2A and 2B data into a single dataset
s2_ds = xr.concat([s2a_ds, s2b_ds], dim='time').sortby('time')

s2_ds

#### Let's have a look at the RGB images in the resulting Dataset

In [ ]:
rgb(s2_ds, col='time')

#### We can use XArray built-in functionality to create an average image along the time dimension:

In [ ]:
# Generate mean composite
s2_mean = s2_ds.mean(dim='time')

# Plot in RGB
rgb(s2_mean, size=10)
plt.title('Temporal mean of the image', fontdict={'fontsize': 20})

#### As you can see the clouds are visible in this mean image. We are now going to remove this clouds using the `fmask` variable, which is included in the data that we have loaded. Let's have a look at the values that it contains and their interpretation:

In [ ]:
s2_ds.fmask.flags_definition

#### We now use these variable to filter the cloud pixels in the image.

In [ ]:
# Identify pixels that are either "valid", "water" or "snow"
cloud_free_mask = (make_mask(s2_ds.fmask, fmask="valid") |
                   make_mask(s2_ds.fmask, fmask="water") |
                   make_mask(s2_ds.fmask, fmask="snow"))

# Apply mask to RGB bands
s2_masked = s2_ds[['nbart_red', 'nbart_green',
                   'nbart_blue']].where(cloud_free_mask)

# Generate mean composite
s2_mean = s2_masked.mean(dim='time')

# Plot in RGB
rgb(s2_mean, size=10)
_ = plt.title('Masking with fmask', fontdict={'fontsize': 20})

#### Unfortunately, as you can see, the `fmask` product does not work very well for Sentinel2 images. As an alternative, there is a better solution with the TSMask product.

#### `TSmask` is a time series cloud and cloud shadow detection algorithm for Sentinel-2 surface reflectance data.  `TSmask` models time series of surface reflectance derived indices and calculates time series abnormality coefficients for pixels in the time series. `TSmask` does not rely on predefined training data to generate complex models with many rule sets, which often work well for data similar to the training data while return poor results for data contrasting to the training data. Instead, `TSmask` identifies cloud and cloud shadows by detecting local abnormalities in temporal and spatial contexts from abnormality coefficients.

#### The `TSmask` analysis algorithm classifies each Sentinel-2 pixel into one of four distinctive categories:

* 0 = No observation
* 1 = Clear
* 2 = Cloud
* 3 = Cloud shadow

#### `TSmask` is available in DEA in a product called `s2_tsmask`. We can inspect its features using `dc.list_measurements()`:

In [ ]:
dc.list_measurements().loc['s2_tsmask']

#### Unlike Sentinel 2 ARD data which is provided in seperate products for Sentinel-2A and 2B, the `s2_tsmask` product is provided as a single dataset that includes classifications for both Sentinel-2A and 2B. Let's load this product using the same query details that used to request the images:

In [ ]:
# Load in temporal cloudmask data
tsmask_ds = dc.load(product='s2_tsmask', 
                    progress_cbk=with_ui_cbk(),
                    **query)

tsmask_ds

#### The dataset has a single varible called `classification`, which contains the cloud and cloud shadow masking data.

#### The `classification` band contains flag information that classifies each pixel into no data, valid, cloudy and cloud shadowed observations:

In [ ]:
tsmask_ds.classification.flags_definition

#### Due to differences in the processing pipelines used to generate Sentinel-2 ARD data and `TSmask`, there may be a different number of observations returned for both of these datasets. The following step can be useful for ensuring that both datasets contain exactly the same observations:

```
matching_times = (tsmask_ds.time - s2_ds.time).time
tsmask_ds = tsmask_ds.sel(time=matching_times)
s2_ds = s2_ds.sel(time=matching_times)
```

#### Now we can plot the same data with a mask generated from `TSmask` applied instead of `fmask`. Note that compared to the example above, the composite is clean and free of cloud or shadow, indicating that classification did a better job of removing these sources of noise from the time series. 

In [ ]:
# Identify pixels that are "Clear"
cloud_free_mask = make_mask(tsmask_ds.classification, classification="valid")

# Apply mask to RGB bands
s2_masked = s2_ds[['nbart_red', 'nbart_green',
                   'nbart_blue']].where(cloud_free_mask)

# Generate mean composite
s2_mean = s2_masked.mean(dim='time')

# Plot in RGB
rgb(s2_mean, size=10)
plt.title('Masking with TSmask', fontdict={'fontsize': 20})

#### Exercise 5.5: Similarly to the previous example. Can you create an averaged image for a city in Australia using the TSmask product.

> Note: Cloud masking is particularly tricky in cities, where certain rooftops present high reflectance values and are often missclassified by cloud detection algorithms. Persistence of high reflectance values over a time series allows to differentiate clouds from high reflectance surfaces.

In [ ]:
# use as many cells as you need

query = {?}

# Load in temporal cloudmask data
tsmask_ds = dc.load(product='s2_tsmask', 
                    progress_cbk=with_ui_cbk(),
                    **query)

# Load in Sentinel-2A and 2B data including fmask cloud masking band
s2a_ds = dc.load(
    product='s2a_ard_granule',
    measurements=['fmask', 'nbart_red', 'nbart_green', 'nbart_blue'],
    progress_cbk=with_ui_cbk(),
    **query)

s2b_ds = dc.load(
    product='s2b_ard_granule',
    measurements=['fmask', 'nbart_red', 'nbart_green', 'nbart_blue'],
    progress_cbk=with_ui_cbk(),
    **query)

# Combine Sentinel-2A and 2B data into a single dataset
s2_ds = xr.concat([s2a_ds, s2b_ds], dim='time').sortby('time')

???
???
???

#### Disclaimer: The original notebook for this tutorial has been taken from the Sandbox `Beginners_guide` folder. Refer to the conditions specified in the original notebook and for updated versions.